IMPORT AND SETTINGS

In [7]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import pandas as pd
import cv2 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import random

try:
    tf.config.experimental.enable_op_determinism()
    print("✅ Op Determinism Abilitato!")
except AttributeError:
    print("⚠️ Attenzione: La tua versione di TF è troppo vecchia per enable_op_determinism.")

def reset_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
SEEDS = [555]

2.21.0-dev20251210
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Op Determinism Abilitato!


DATASET LOADING AND MODELLING

In [8]:
def load_dataset():
    folder='dataset/images'
    data=[]
    for filename in sorted(os.listdir(folder)):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path) #opencv save in bgr
        data.append({
            'image':img,
            'filename':filename
        })

    print(len(data),'images loaded')
    print('file name is: ',data[0]['filename'], 'shape of the image is:  ', data[0]['image'].shape )

    label=pd.read_csv('dataset/raw/bbx_annotations.csv')
    print(label.shape, label.iloc[0]['filename'])
    #images order is random, and for 1 image you can have more class

    print('we have', len(label['class'].unique()), 'different classes')

    #replace biggger img with half sized ones
    #cv2.imwrite('resize_image/last_img_pre_downsampling.jpg',data[-100]['image'])
    for i,item in enumerate(data):
        if "upper" in item["filename"].lower():
            data[i]['image']=cv2.resize(
                data[i]['image'],
                (data[i]['image'].shape[1]//2,data[i]['image'].shape[0]//2)
                ,interpolation=cv2.INTER_AREA
            )
    #cv2.imwrite('resize_image/last_img_post_downsampling.jpg',data[-100]['image'])
       
    return data, label

FROM THE PURE DATASET TO THE TRAIN AND TEST DATA AND LABEL

In [9]:
def dataset_modelling(dataset,annotation):   
    dataset_df = pd.DataFrame(dataset) 
    label_map={'goalpost':0,
               'ball':1,
               'robot':2,
               'goalspot':3,
               'centerspot':4}
    def get_vector(classes_found):
        vec=np.zeros(5,dtype=int)

        for c in classes_found:
            if c in label_map:
                vec[label_map[c]]=1
        return list(vec)
    
    grouped = annotation.groupby('filename')['class'].apply(list).reset_index()
    grouped['label']=grouped['class'].apply(get_vector)
    final_annotation=grouped[['filename','label']]

    final_dataset= pd.merge(dataset_df, final_annotation[['filename', 'label']], on='filename', how='inner')
    final_dataset.to_csv('csv/temp/final_dataset.csv')
    final_dataset=final_dataset.drop(columns=['filename'])
    df_train, df_test = train_test_split(final_dataset, test_size=0.2, random_state=42)
    x_train = np.array(df_train['image'].tolist()).astype('float32') /255.0
    y_train = np.array(df_train['label'].tolist()).astype('float32')
    
    x_test = np.array(df_test['image'].tolist()).astype('float32') / 255.0
    y_test = np.array(df_test['label'].tolist()).astype('float32')
    return x_train, y_train,x_test,y_test
    

DOUBLING THE DATA 

In [10]:
def augment_train_set(x_train,y_train):

    rng = np.random.RandomState(42)    
    
    x_flipped = np.flip(x_train, axis=2)
    y_flipped = y_train
    #---------------------------------#
    noise = rng.normal(loc=0.0, scale=0.05, size=x_train.shape)
    x_noisy = x_train + noise
    x_noisy = np.clip(x_noisy, 0., 1.)
    y_noise = y_train
    #--------------------------------#
    factors = rng.uniform(0.1, 0.3, size=(x_train.shape[0], 1, 1, 1))
    x_train_bright = np.clip(x_train + factors, 0.0, 1.0)
    #---------------------------------#
    contrast_fact = rng.uniform(0.8, 1.5, size=(x_train.shape[0], 1, 1, 1))
    mean = np.mean(x_train, axis=(1, 2), keepdims=True)
    x_contrast = (x_train - mean) * contrast_fact + mean
    x_contrast = np.clip(x_contrast, 0.0, 1.0)
    #--------------------------------#
    x_train_aug = np.concatenate([x_train,x_flipped, x_noisy,x_train_bright,x_contrast], axis=0)
    y_train_aug = np.concatenate([y_train,y_train, y_train,y_train,y_train], axis=0)

    #avoid to have all noisy data in validation--> shuffle
    indices = np.arange(x_train_aug.shape[0])
    rng.shuffle(indices)
    x_train_aug = x_train_aug[indices]
    y_train_aug = y_train_aug[indices]

    return x_train_aug, y_train_aug

MODEL BUILIDNG, 
kernel dimesnsion, pooling dimension, fc layers dimension, number of conv layer and learning rate have different combination.
instead, i fixed:
pooling stride=2 
pooling type: avg pooling
number of kernel per layer: 16, 32, 64...
last pooling: glob avg pool

In [11]:
def build_model():
    model=models.Sequential()
    model.add(tf.keras.Input(shape=(240,320,3)))

    for i in range(4):
        kernel_number=16*(2**i)
        model.add(layers.Conv2D(kernel_number,(7,7),activation='relu',padding='same'))
        model.add(layers.AveragePooling2D((3,3),strides=2,padding='same'))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(5,activation='sigmoid'))

    #model.summary()
    return model


MAIN

In [ ]:
dataset, annotation=load_dataset()
x_train,y_train, x_test, y_test=dataset_modelling(dataset, annotation)
x_train,y_train=augment_train_set(x_train,y_train)

for seed in SEEDS:
    bestf1=0
    all_results = []

    print(f"\n ================== INIZIO CICLO CON SEED: {seed} ================== ")

    
    tf.keras.backend.clear_session()
    reset_seeds(seed)
    model=build_model()

    opt=tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=[tf.keras.metrics.Precision(name='precision'),
                        tf.keras.metrics.Recall(name='recall'),
                        ])
    early_stop=EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
    )
    reset_seeds(seed)
    #class_weights_dict=compute_class_weight(y_train)
    history=model.fit(
        x_train,y_train,
        epochs=120,
        batch_size=16, 
        validation_split=0.2, 
        callbacks=[early_stop]
        )

    y_pred=model.predict(x_test)
    predictions_binary = (y_pred > 0.5).astype(int)
    target_names = ['goalpost','ball','robot','goalspot','centerspot']
    report_dict = classification_report(y_test, predictions_binary, target_names=target_names, output_dict=True)
    f1_macro = report_dict['macro avg']['f1-score']
    

    df_report = pd.DataFrame(report_dict).transpose()
    df_report = df_report.round(2)
    df_report['support'] = df_report['support'].astype(int)
    csv_path = f'csv/report/report_data_aug_fnbctry_{seed}.csv'
    df_report.to_csv(csv_path)




2452 images loaded
file name is:  lower_100056_jpg.rf.ec9852c66b4eee4a185317210a378f16.jpg shape of the image is:   (240, 320, 3)
(8125, 8) upper_604302_jpg.rf.6215ee30a829ec658154eb4d067dfdf5.jpg
we have 5 different classes


FEATURES MAP

In [ ]:
def visualize_feature_maps(model, input_image):
    layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]
    layer_names = [layer.name for layer in model.layers if 'conv' in layer.name]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    img_batch = np.expand_dims(input_image, axis=0)
    activations = activation_model.predict(img_batch)
    images_per_row = 16
    for layer_name, layer_activation in zip(layer_names, activations):
        n_features = layer_activation.shape[-1] 
        size_h = layer_activation.shape[1]     
        size_w = layer_activation.shape[2]
        n_features = min(n_features, 32) 
        n_cols = n_features // images_per_row
        
        if n_cols == 0: n_cols = 1 
        
        display_grid = np.zeros((size_h * n_cols, images_per_row * size_w))
        
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0, :, :, col * images_per_row + row]
                
                channel_image -= channel_image.mean()
                channel_image /= (channel_image.std() + 1e-5) 
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                
                display_grid[col * size_h : (col + 1) * size_h,
                             row * size_w : (row + 1) * size_w] = channel_image

        # Plotting
        scale = 1. / size_h
        plt.figure(figsize=(scale * display_grid.shape[1],
                            scale * display_grid.shape[0]))
        plt.title(f"Layer: {layer_name} ({n_features} feature maps)")
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        plt.savefig('images/feature_map/data_aug_fnbC.jpg', bbox_inches='tight', dpi=150)
        plt.show()
index_ball = np.where(y_test[:, 1] == 1)[0][0] 
img_to_test = x_test[index_ball]

visualize_feature_maps(model, img_to_test)

